<center>
    <h1>INF285 - Computación Científica</h1>
    <h2>Tarea 0: <i>Hands-on Jupyter Notebook</i></h2>
    <h3>Anghelo Carvajal <br> 201473062-4 <br> anghelo.carvajal.14@sansano.usm.cl</h3>
</center>


In [1]:
import numpy as np
from numpy import pi as π
from numpy import e
from scipy.sparse import linalg as sparse
from scipy import linalg
import matplotlib.pyplot as plt
import ipywidgets as ipyw

## Pregunta 1.- Valores y vectores propios

### 1.A Construya una función en python que obtenga numéricamente el $k$-ésimo valor propio y vector propio de $T_n$, donde se considera que los valores propios fueron ordenados de menor a mayor.

Primero que nada, construimos 2 funciones que nos ayudaran, uno que construya la matriz $T_n$ (`matrix_t_n(n)`), y otra que calcule los valores propios $\lambda_k$ (`eigen_value(n, k)`) en base a la ecuacion entregada

In [2]:
def matrix_t_n(n):
    matrix = np.zeros((n, n), dtype=np.float64)
    for x in range(len(matrix)):
        for y in range(len(matrix[0])): # No era necesario el segundo for, pero ya lo hice...
            if x == y:
                matrix[x][y] = 2
            if x + 1 == y or x - 1 == y:
                matrix[x][y] = -1
    return matrix

def eigen_value(n, k):
    assert n >= 2
    assert 1 <= k and k <= n
    return 2 - 2*np.cos((k-1)*π/(n+1))


In [3]:
'''
Input: 
n - (integer) Dimension of the matrix T_n, n>=2.
k - (integer) 1<=k<=n.

Output:
lambda_k - (double) k-th eigenvalue
v_k - (numpy array) k-th eigenvector
'''
def eigs_T_n(n, k):
    lambda_k = eigen_value(n, k)
    matrix = matrix_t_n(n)
    v_k = sparse.eigs(matrix, k=1, sigma=lambda_k)[1]
    return [lambda_k, v_k]

### 1.2 Realice un plot de ```eigs_T_n(n, k)``` considerando $k=4$. ¿Que observa?

Simplemente se usan las funciones `scatter()` e `interact()` de `matplotlib` y `ipywidgets` respectivamente. 

In [4]:
def plot_eigs_T_n(n, k):
    plt.scatter(np.arange(n), eigs_T_n(n, k)[1])
    plt.show()

ipyw.interact(plot_eigs_T_n, n=(10, 1000),k=ipyw.fixed(4));

interactive(children=(IntSlider(value=505, description='n', max=1000, min=10), Output()), _dom_classes=('widge…

Observo que al variar el valor de $n$, la grafica va tendiendo a parecerse a una funcion sinusoidal.

## Pregunta 2.- A matrix equation

Considere la siguiente ecuación matricial:
\begin{equation*}
\frac{(n-1)^2}{4}(T_n\,U_n+U_n\,T_n)+9\,U_n=\,F_n\in \mathbb{R}^{n\times n}
\end{equation*}

### 2.1 Construya una función en python que obtenga $U_n$ dado $n$

Primero se construye una funcion que sea capaz de generar la matriz $F_n$ (`matrix_f_n()`).

In [5]:
def x_j(n, j):
    return -1+(j-1)*(2/(n-1))

def y_i(n, i):
    return -1+(i-1)*(2/(n-1))
                     
def matrix_f_n(n):
    matrix = np.zeros((n, n), dtype=np.float64)
    for i in range(0, n):
        for j in range(0, n):
            matrix[i][j] = e**(-10*((y_i(n, i+1)-1)**2+(x_j(n, j+1)-1/2)))
    return matrix


Ahora necesitamos construir la funcion encargada de encontrar la matriz $U_n$.

Como la ecuacion pedida se parece a una ecuacion de Sylverster ($AX + XB = C$), desarrollamos un poco el algebra, con el objetivo de acercarnos a eso:
\begin{align*}
\frac{(n-1)^2}{4}(T_n U_n + U_n T_n)+9 U_n &= F_n \\
\frac{(n-1)^2}{4} T_n U_n + \frac{(n-1)^2}{4} U_n T_n + 9 U_n &= F_n \\
\frac{(n-1)^2}{4} T_n U_n + U_n\left(\frac{(n-1)^2}{4} T_n + 9 I_n\right) &= F_n
\end{align*}

De la ultima ecuacion, podemos ver claramente que es la ecuacion de Sylverster, donde:
\begin{align*}
A &= \frac{(n-1)^2}{4} T_n \\
B &= \left(\frac{(n-1)^2}{4} T_n + 9 I_n\right) \\
C &= F_n \\
X &= U_n \\
\end{align*}

Sabiendo lo anterior, para poder obtener la matriz $U_n$ usaremos la funcion `solve_sylvester()`.

In [6]:
'''
Input: 
n - (integer) Dimension of the problem, n>=2.
Output:
U_n - (2D-array numpy array) The solution of the problem.
'''
def find_U_n(n):
    assert n >= 2
    t_n = matrix_t_n(n)
    f_n = matrix_f_n(n)
    A = (((n-1)**2)/4)*t_n
    B = ((((n-1)**2)/4)*t_n + 9 * np.identity(n))
    U_n = linalg.solve_sylvester(A, B, f_n)
    return U_n


### 2.2 Realice un heatmap de $U_n(n)$ considerando un widget para variar $n\in\{10,\dots,50\}$. ¿Qué observa?

In [7]:
def heatmap_u_n(n):
    plt.imshow(find_U_n(n), cmap='hot', interpolation='nearest')
    plt.show()
    return

ipyw.interact(heatmap_u_n, n=(10, 50))

interactive(children=(IntSlider(value=30, description='n', max=50, min=10), Output()), _dom_classes=('widget-i…

<function __main__.heatmap_u_n(n)>

Observo que el heatmap se concentra en la esquina inferior izquierda. Ademas, al tener $n$ mas grandes, el grafico tiende a ser mas detallado y fino, al contrario que con los $n$ mas pequeños.

## Referencias

 * [np.zeros](https://docs.scipy.org/doc/numpy/reference/generated/numpy.zeros.html)
 * [sparse.eigs](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.eigs.html)
 * [Plotting a list of (x, y) coordinates in python matplotlib](https://stackoverflow.com/questions/21519203/plotting-a-list-of-x-y-coordinates-in-python-matplotlib)
 * [linalg.solve_sylvester](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.linalg.solve_sylvester.html)
 * [Plotting a 2D heatmap with Matplotlib](https://stackoverflow.com/questions/33282368/plotting-a-2d-heatmap-with-matplotlib)
 * [interact](https://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html)